In [1]:
import pandas as pd
import sqlite3
import warnings
import os
warnings.filterwarnings("ignore")


In [2]:
customer = pd.read_csv("customer.csv")
customer.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes


In [3]:
cust_loc= pd.read_csv("cust_loc.csv")
cust_loc.head()

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [54]:
cust_services=pd.read_csv("cust_services.csv")
cust_services.head()

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [58]:
cust_account= pd.read_csv("cust_account.csv")
cust_account.head()

,Account_id,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges
0,3668-QPYBK,2,Month-to-month,Mailed check,Yes,53.85,108.15
1,9237-HQITU,2,Month-to-month,Electronic check,Yes,70.70,151.65
2,9305-CDSKC,8,Month-to-month,Electronic check,Yes,99.65,820.5
3,7892-POOKP,28,Month-to-month,Electronic check,Yes,104.80,3046.05
4,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3


In [59]:
cust_churn= pd.read_csv("cust_churn.csv")
cust_churn.head()

,Id,Churn
0,3668-QPYBK,Yes
1,9237-HQITU,Yes
2,9305-CDSKC,Yes
3,7892-POOKP,Yes
4,0280-XJGEX,Yes


# Creating Database and Tables

In [155]:
conn= sqlite3.connect("churn.db")
cur=conn.cursor()

In [9]:
customer.to_sql("customer",conn, if_exists="replace",index=False)
cust_loc.to_sql("cust_loc",conn, if_exists="replace",index=False)
cust_services.to_sql("cust_services",conn, if_exists="replace",index=False)
cust_account.to_sql("cust_account",conn, if_exists="replace",index=False)
cust_churn.to_sql("cust_churn",conn, if_exists="replace",index=False)

In [8]:
def table_info(conn,cur):
    """
    prints out all columns for every table in the DB
    """
    tables = cur.execute("select name from sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name= table_name[0]
        table= pd.read_sql_query("SELECT * FROM {} limit 0 ".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print("\t"+col)
            print()
table_info(conn,cur)

customer
	CustomerID

	Gender

	SeniorCitizen

	Partner

	Dependents

cust_loc
	Cust_ID

	State

	Latitude

	Longitude

	ZipCode

cust_services
	Cust_ID

	PhoneService

	MultipleLines

	InternetService

	OnlineSecurity

	OnlineBackup

	DeviceProtection

	TechSupport

	StreamingTV

	StreamingMovies

cust_account
	Account_id

	Tenure

	Contract

	PaymentMethod

	PaperlessBilling

	MonthlyCharges

	TotalCharges

cust_churn
	Id

	Churn



# Analysis on SQL

In [9]:
pd.read_sql("select Gender, count(Gender) from customer group by Gender;",conn)

,Gender,count(Gender)
0,Female,3488
1,Male,3555


In [10]:
pd.read_sql("select Partner, count(Partner) from customer group by Partner;",conn)


,Partner,count(Partner)
0,No,3641
1,Yes,3402


In [11]:

pd.read_sql("select Dependents, count(Dependents) from customer group by Dependents;",conn)

,Dependents,count(Dependents)
0,No,5416
1,Yes,1627


In [12]:

pd.read_sql("select round(avg(Tenure), 2) from cust_account;",conn)


,"round(avg(Tenure), 2)"
0,32.37


In [13]:
pd.read_sql("select round(avg(MonthlyCharges), 2) from cust_account;",conn)


,"round(avg(MonthlyCharges), 2)"
0,64.76


In [14]:

pd.read_sql("select round(avg(TotalCharges), 2) from cust_account;",conn)

,"round(avg(TotalCharges), 2)"
0,2279.73


In [15]:
pd.read_sql("select count(*) from cust_account where TotalCharges > 	(Select avg(TotalCharges) from cust_account);",conn)

,count(*)
0,4922


In [16]:
pd.read_sql("select count(*) from cust_account where TotalCharges < 	(Select avg(TotalCharges) from cust_account);",conn)


,count(*)
0,2121


In [17]:
pd.read_sql("select PaymentMethod, count(PaymentMethod) from cust_account group by PaymentMethod;",conn)


,PaymentMethod,count(PaymentMethod)
0,Bank transfer (automatic),1544
1,Credit card (automatic),1522
2,Electronic check,2365
3,Mailed check,1612


In [18]:
pd.read_sql("select Contract, count(Contract) from cust_account group by Contract;",conn)

,Contract,count(Contract)
0,Month-to-month,3875
1,One year,1473
2,Two year,1695


In [19]:
pd.read_sql("select Churn, count(Churn) from cust_churn group by Churn;",conn)

,Churn,count(Churn)
0,No,5174
1,Yes,1869


In [24]:
pd.read_sql("""select cust_churn.Churn,  avg(cust_account.TotalCharges) 
from cust_churn
inner join cust_account on cust_account.Account_Id = cust_churn.ID
group by cust_churn.Churn;""",conn)

,Churn,avg(cust_account.TotalCharges)
0,No,2549.911442
1,Yes,1531.796094


In [25]:
pd.read_sql("""select cust_churn.Churn,  avg(cust_account.MonthlyCharges) 
from cust_churn
inner join cust_account on cust_account.Account_Id = cust_churn.ID
group by cust_churn.Churn;""",conn)

,Churn,avg(cust_account.MonthlyCharges)
0,No,61.265124
1,Yes,74.441332


In [34]:
pd.read_sql("""select customer.Gender, cust_churn.Churn,  count(customer.Gender) 
from cust_churn
inner join customer on customer.CustomerID = cust_churn.ID
group by cust_churn.Churn, customer.Gender;""",conn)

,Gender,Churn,count(customer.Gender)
0,Female,No,2549
1,Male,No,2625
2,Female,Yes,939
3,Male,Yes,930


In [37]:
pd.read_sql("""select cust_churn.Churn,cust_account.contract,  count(cust_account.Contract) 
from cust_churn
inner join cust_account on cust_account.Account_Id = cust_churn.ID
group by cust_churn.Churn,contract;""",conn)

,Churn,Contract,count(cust_account.Contract)
0,No,Month-to-month,2220
1,No,One year,1307
2,No,Two year,1647
3,Yes,Month-to-month,1655
4,Yes,One year,166
5,Yes,Two year,48


# Joining all tables

In [38]:
cur.execute(
"""
ALTER TABLE cust_loc
RENAME COLUMN Cust_ID TO CustomerID;
""")

In [39]:
table_info(conn,cur)

customer
	CustomerID

	Gender

	SeniorCitizen

	Partner

	Dependents

cust_loc
	CustomerID

	State

	Latitude

	Longitude

	ZipCode

cust_services
	Cust_ID

	PhoneService

	MultipleLines

	InternetService

	OnlineSecurity

	OnlineBackup

	DeviceProtection

	TechSupport

	StreamingTV

	StreamingMovies

cust_account
	Account_id

	Tenure

	Contract

	PaymentMethod

	PaperlessBilling

	MonthlyCharges

	TotalCharges

cust_churn
	Id

	Churn



In [40]:
cur.execute(
"""
SELECT 
*
FROM customer
INNER JOIN cust_loc
USING (CustomerID)
""")
cur.fetchone()

('3668-QPYBK',
 'Male',
 'No',
 'No',
 'No',
 'California',
 33.964131,
 -118.272783,
 90003)

In [41]:
cust_loc.head()

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [44]:
cust_loc.rename(columns={"Cust_ID": "CustomerID"},inplace=True)

In [45]:
cust_loc.head()

,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [46]:
pd.concat([customer,cust_loc], join="inner", axis=1)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,Male,No,No,No,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,Female,No,No,Yes,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,Female,No,No,Yes,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,Female,No,Yes,Yes,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,Male,No,No,Yes,0280-XJGEX,California,34.039224,-118.266293,90015
...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Female,No,No,No,2569-WGERO,California,34.341737,-116.539416,92285
7039,6840-RESVB,Male,No,Yes,Yes,6840-RESVB,California,34.667815,-117.536183,92301
7040,2234-XADUH,Female,No,Yes,Yes,2234-XADUH,California,34.559882,-115.637164,92304
7041,4801-JZAZL,Female,No,Yes,Yes,4801-JZAZL,California,34.167800,-116.864330,92305


In [55]:
cust_services.head()

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [56]:
cust_services.rename(columns={"Cust_ID": "CustomerID"})

,CustomerID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
7039,6840-RESVB,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes
7040,2234-XADUH,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
7041,4801-JZAZL,No,No phone service,DSL,Yes,No,No,No,No,No


In [60]:
cust_account.rename(columns={"Account_id": "CustomerID"},inplace=True)
cust_churn.rename(columns={"Id": "CustomerID"},inplace=True)

In [61]:
dfs_to_join = [customer, cust_loc, cust_services, cust_account, cust_churn]

In [62]:
customer.loc[customer["Gender"] == "Male",]

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
4,0280-XJGEX,Male,No,No,Yes
6,8779-QRDMV,Male,Yes,No,No
7,1066-JKSGK,Male,No,No,No
8,6467-CHFZW,Male,No,Yes,Yes
...,...,...,...,...,...
7034,3605-JISKB,Male,Yes,Yes,No
7035,9767-FFLEM,Male,No,No,No
7036,8456-QDAVC,Male,No,No,No
7039,6840-RESVB,Male,No,Yes,Yes


In [63]:
churn_all = pd.concat(dfs_to_join, join="inner", axis=1)

In [64]:
churn_all.columns.duplicated()

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False])

In [65]:
churn_all = churn_all.loc[:, ~churn_all.columns.duplicated()]

In [66]:
churn_all.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,Cust_ID,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,3668-QPYBK,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,9237-HQITU,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,9305-CDSKC,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.5,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,7892-POOKP,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,0280-XJGEX,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3,Yes


In [67]:
churn_all.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
Cust_ID              object
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [69]:
churn_all.to_sql("churn_all",conn, if_exists="replace",index=False)

# Analysis of Lifetime Value

- LTV is the total worth of a customer to a business over the entire duration of their relationship.
- It is an important metric as it costs less to keep existing customers than it does to acquire new ones; 
- so, increasing the value of your existing customers is an excellent way to drive growth.

## For the customers who unsubscribed what was their average LTV? How long did they usually stay in their service?

In [71]:
## Extract those who unsubscribed the serviced (Churn = Yes)

churn_df= pd.read_sql("select * from churn_all where churn='Yes'",conn)

In [74]:
pd.read_sql("SELECT sum(TotalCharges) from churn_all",conn)

,sum(TotalCharges)
0,16056168.7


In [78]:
## Examining total charges

pd.to_numeric(churn_df["TotalCharges"]).describe()

count    1869.000000
mean     1531.796094
std      1890.822994
min        18.850000
25%       134.500000
50%       703.550000
75%      2331.300000
max      8684.800000
Name: TotalCharges, dtype: float64

We can see that around 20% of the TotalCharges are very high, so let's divide the data to see each distribution.

In [81]:
churn_df["TotalCharges"]=churn_df.TotalCharges.astype(float)

In [82]:
# Find the 80th percentile of the data in TotalCharges
churn_df.TotalCharges.quantile(0.8)

2840.4100000000003

In [83]:
churn_df.query("TotalCharges <= 2840.41")

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,Cust_ID,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,3668-QPYBK,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,9237-HQITU,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,9305-CDSKC,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
5,4190-MFLUW,Female,No,Yes,No,California,34.066367,-118.309868,90020,4190-MFLUW,...,Yes,No,No,10,Month-to-month,Credit card (automatic),No,55.20,528.35,Yes
6,8779-QRDMV,Male,Yes,No,No,California,34.023810,-118.156582,90022,8779-QRDMV,...,No,No,Yes,1,Month-to-month,Electronic check,Yes,39.65,39.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,1122-JWTJW,Male,No,Yes,Yes,California,32.698964,-115.886656,92259,1122-JWTJW,...,No,No,No,1,Month-to-month,Mailed check,Yes,70.65,70.65,Yes
1864,1699-HPSBG,Male,No,No,No,California,33.745746,-116.514215,92264,1699-HPSBG,...,Yes,Yes,No,12,One year,Electronic check,Yes,59.80,727.80,Yes
1865,8775-CEBBJ,Female,No,No,No,California,32.790282,-115.689559,92273,8775-CEBBJ,...,No,No,No,9,Month-to-month,Bank transfer (automatic),Yes,44.20,403.35,Yes
1866,6894-LFHLY,Male,Yes,No,No,California,34.264124,-114.717964,92280,6894-LFHLY,...,No,No,No,1,Month-to-month,Electronic check,Yes,75.75,75.75,Yes


In [84]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
total_charges_under80 = churn_df.query("TotalCharges <= 2840.41")
total_charges_above80 = churn_df.query("TotalCharges > 2840.41")

In [85]:
# Show the distribution of people under the 80th percentile
total_charges_under80.TotalCharges.describe()

count    1495.000000
mean      711.265819
std       766.848197
min        18.850000
25%        85.025000
50%       371.650000
75%      1128.225000
max      2839.650000
Name: TotalCharges, dtype: float64

In [86]:
# Show the distribution of people above the 80th percentile
total_charges_above80.TotalCharges.describe()

count     374.000000
mean     4811.723262
std      1436.724288
min      2841.550000
25%      3522.462500
50%      4571.100000
75%      5891.212500
max      8684.800000
Name: TotalCharges, dtype: float64

In [87]:
total_charges_under80.Tenure.describe()

count    1495.000000
mean        9.935117
std        10.742349
min         1.000000
25%         1.000000
50%         6.000000
75%        15.000000
max        61.000000
Name: Tenure, dtype: float64

In [88]:
total_charges_above80.Tenure.describe()

count    374.000000
mean      50.133690
std       12.334841
min       27.000000
25%       40.000000
50%       49.500000
75%       60.000000
max       72.000000
Name: Tenure, dtype: float64

The average LTV of the 80% of those who unsubscribed is \\$ 711, and their tenure is near 10 months. On the hand, the average LTV of the 20 % of those who unsubscribed is $ 4811, and their tenure is near 50 months.  

## 2. What kinds of services they subscribed when they were still a customer?

In [90]:
total_charges_under80.PhoneService.value_counts()

Yes    1334
No      161
Name: PhoneService, dtype: int64

In [91]:
total_charges_under80.PhoneService.value_counts() / total_charges_under80.shape[0]

Yes    0.892308
No     0.107692
Name: PhoneService, dtype: float64

In [92]:
total_charges_under80.shape

(1495, 26)

In [93]:
# Above 80 (percentage)
total_charges_above80.PhoneService.value_counts() / total_charges_above80.shape[0]

Yes    0.975936
No     0.024064
Name: PhoneService, dtype: float64

For the top 20% (high LTV) only 2% of them didn't use the phone service. On the other hand, lower 80% (LTV) had around 11% of the people that didn't use the phone service. That is around 5 times more as the top 20%. Apparently if you are not subscribed to the phone service you are more likely to be in the bottom 80% of the LTV. We as a company can try to sell this service to more people.

In [94]:
total_charges_under80_use_phone = total_charges_under80.query("PhoneService == 'Yes'")

total_charges_under80_use_phone.MultipleLines.value_counts() / total_charges_under80_use_phone.shape[0]

No     0.590705
Yes    0.409295
Name: MultipleLines, dtype: float64

In [95]:
total_charges_above80_use_phone = total_charges_above80.query("PhoneService == 'Yes'")

total_charges_above80_use_phone.MultipleLines.value_counts() / total_charges_above80_use_phone.shape[0]

Yes    0.832877
No     0.167123
Name: MultipleLines, dtype: float64

For the top 20% of LTV who used the phone service 84% of them used multiple lines, which is 2 times as much as the proportion of people in the low 80% LTV that is around 40%. Having multiple lines is going to get you closer to be in the top 20% of the LTV.

In [96]:
# Under 80 (percentage)
total_charges_under80.InternetService.value_counts() / total_charges_under80.shape[0]

Fiber optic    0.640803
DSL            0.283612
No             0.075585
Name: InternetService, dtype: float64

In [97]:
# Above 80 (percentage)
total_charges_above80.InternetService.value_counts() / total_charges_above80.shape[0]

Fiber optic    0.906417
DSL            0.093583
Name: InternetService, dtype: float64

All the people in the high 20% LTV used internet service. On the other hand 8% of the people in the low 80th percentile of the LTV did not have internet connection. Also, in the top 20% (high LTV) 90% of them had fiber optic, in comparison to only 64% in the low LTV. So as a company if we want to increase the LTV of customers, we should recommend using internet and Fiber Optic connection.

In [98]:
import numpy as np

In [99]:
total_charges_under80_use_internet = total_charges_under80.query('InternetService!="No"')
proportion_internet_sub_service_under80 = np.array([total_charges_under80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0]])

In [100]:
proportion_internet_sub_service_under80

array([0.1316932 , 0.13603473, 0.22720695, 0.23516643, 0.37698987,
       0.3813314 ])

In [101]:
total_charges_above80_use_internet = total_charges_above80.query('InternetService!="No"')
proportion_internet_sub_service_above80 = np.array([total_charges_above80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0]])

In [102]:
proportion_internet_sub_service_above80

array([0.30213904, 0.32620321, 0.55882353, 0.58823529, 0.78342246,
       0.77807487])

In the top 20% LTV, both streaming movies and streaming tv percentages are around 80%, in comparison in the low 80% LTV they are around 40%. Device protection and online backup for the top 20% LTV are around 55%, in comparison to only around 22% in the low LTV. As a company we would like to sell to customers streaming TV and movies and also online backup and device protection if we want to increase LTV.

## 3. For those who churned what is the proportion of each kind of contract?

In [104]:
# Under 80 (percentage)
total_charges_under80.Contract.value_counts() / total_charges_under80.shape[0]

Month-to-month    0.959197
One year          0.036789
Two year          0.004013
Name: Contract, dtype: float64

In [105]:
# Above 80 (percentage)
total_charges_above80.Contract.value_counts() / total_charges_above80.shape[0]

Month-to-month    0.590909
One year          0.296791
Two year          0.112299
Name: Contract, dtype: float64

In [156]:
## Extract those who unsubscribed the services (Churn = Yes)

paying_df = pd.read_sql("select * from churn_all where Churn = 'No'", conn)

In [157]:
paying_df.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
Cust_ID              object
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [152]:
paying_df["TotalCharges"]=paying_df.TotalCharges.astype(float)

ValueError: could not convert string to float: ''

In [158]:
paying_df.TotalCharges.describe()

count     5174
unique    4966
top           
freq        11
Name: TotalCharges, dtype: object

In [161]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
paying_total_charges_under80 = paying_df.query("float(TotalCharges) <= '4895.8'")
paying_total_charges_above80 = paying_df.query("float(TotalCharges) > '4895.8'")

ValueError: "float" is not a supported function